# 📽넷플릭스 대한민국 분석과 시각화 그리고 추천 시스템 구상

# 1_목표


이 데이터셋은 전세계의 데이터가 포함된 데이터셋인데, 국가를 대한민국으로 한정지은 다음...
```
1. 데이터 분석(EDA)
2. 시각화
3. Baseline 모델 생성 후 추천 알고리즘 구현
```
과 같은 순서로 진행할 것이다.

## 1_1_데이터 분석(EDA)

In [ ]:
import pandas as pd
import numpy as np

import datetime 
import time

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
netflix_df = pd.read_csv('dataset/netflix_titles.csv',
                         encoding='utf-8', engine='python')

print(netflix_df.shape)

In [ ]:
netflix_df.head()

In [ ]:
for col in netflix_df.columns:
    print("{}\n".format(netflix_df[col].head()))

주목해야할 점은 'country' 컬럼이다. 이 데이터셋은 전세계의 모든 데이터가 포함되어 있고, 우리가 봐야할 것은 국가를 한정시키고 분석하는 것이 이번 목표이기 때문에 `netflix_rok_df`라는 이름으로 새롭게 생성하도록 한다.

In [ ]:
netflix_rok_df = netflix_df[netflix_df['country'] == 'South Korea']

print(netflix_rok_df.shape)

데이터 수가 적은 것 같아서 걱정되지만, 차후에 크롤링을 통해 데이터를 추가하는 작업도 고민해 볼 필요가 있을 것 같다.

일단은 계속 진행해 보겠다.

In [ ]:
netflix_rok_df.count()

In [ ]:
netflix_rok_df.info()

In [ ]:
netflix_rok_df.isnull().sum()

### 수치형 / 범주형 변수

In [ ]:
num_cols = [col for col in netflix_rok_df.columns if netflix_rok_df[col].dtype in ["int64", "float64"]]
netflix_rok_df[num_cols].describe()

In [ ]:
cat_cols = [col for col in netflix_rok_df.columns if netflix_rok_df[col].dtype in ['O']]
netflix_rok_df[cat_cols].describe()

In [ ]:
for col in cat_cols:
    uniq = np.unique(netflix_rok_df[col].astype(str))
    print('# 컬럼명: {}, 고유값의 개수: {}, 고유값 목록: {}'.format(col, len(uniq), uniq))

`netflix_rok_df`의 'type' 컬럼이 속한 고유값을 `.unique()` 함수로 확인해본다.

In [ ]:
netflix_rok_df['type'].unique()

In [ ]:
netflix_rok_df.dtypes

`netflix_rok_df`의 'type' 컬럼 중 'TV Show' 값을 `netflix_rok_shows`라는 이름으로 추출한다.

In [ ]:
netflix_rok_shows = netflix_rok_df[netflix_rok_df['type'] == 'TV Show']
netflix_rok_shows.head()

In [ ]:
netflix_rok_shows.dtypes

`netflix_rok_df`의 'type' 컬럼 중 'Movie' 데이터를 `netflix_rok_movies`라는 이름으로 추출한다.

In [ ]:
netflix_rok_movies = netflix_rok_df[netflix_rok_df['type'] == 'Movie']
netflix_rok_movies.head()

# 2_시각화

시각화 작업에 앞서, 기존의 "matplotlib", "seaborn"과 같은 시각화 도구 외에도 "Plotly express"라는 시각화 도구가 존재한다.

사용법은 seaborn과 크게 다르지 않기 때문에 개인적으로 이쪽을 권장하고 싶다.

설치 방법은 'Anaconda Prompt' 창에서 `conda install` ? `pip install` ? 명령어를 통해서 "plotly" & "plotly Express" 를 설치를 진행하면 된다.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf

cf.go_offline()

## 2_1_넷플릭스 대한민국 컨텐츠 중 한국 영화와 TV 프로그램이 차지하는 비중

먼저 'type' 컬럼의 'Movie' 데이터와 'TV Show' 데이터가 차지하는 비중을 간단하게 시각화해보겠다.

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
ax.set_title('넷플릭스 대한민국 한국 영화 Vs. TV 프로그램', family='D2Coding', size=20)

plt.xkcd()
    
sns.set(style='whitegrid')
sns.countplot(data=netflix_rok_df, x='type', palette='pastel')

이 데이터셋에는 한국 영화보다는 한국 TV 프로그램의 비중이 더 많은 것으로 확인되었다.

## 2_2_넷플릭스 대한민국 컨텐츠 추가 시기 확인

### 2_2_1_TV 프로그램이 추가된 시기 확인

다음으로 TV 프로그램이 어느 연도에 그리고 어느 월에 많이 추가되었는지를 plotly의 Heatmap을 통해 시각화해본다.

In [ ]:
netflix_shows_date = netflix_rok_shows[['date_added']].dropna()
netflix_shows_date['year'] = netflix_shows_date['date_added'].apply(lambda x : x.split(', ')[-1])
netflix_shows_date['month'] = netflix_shows_date['date_added'].apply(lambda x : x.lstrip().split(' ')[0])

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'][::-1]
df = netflix_shows_date.groupby('year')['month'].value_counts().unstack().fillna(0)[month_order]

fig = px.imshow(df, labels=dict(color='Count'), x=df.columns, y=df.index)
fig.update_layout(title='넷플릭스 대한민국 TV 프로그램 업데이트 연도별 월간 추가 추세')

fig.show()

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
ax.set_title('넷플릭스 대한민국 TV 프로그램 연간 추가 추세', family='D2Coding', size=20)
    
plt.xkcd()

sns.set(style='whitegrid')
sns.countplot(data=netflix_rok_shows, y='release_year',
              palette='pastel', order=netflix_rok_shows['release_year']
              .value_counts().index[0:])

2019년 10월의 색깔이 가장 진한 것으로 보아 많은 양의 TV 프로그램이 추가가 된 것을 알 수 있다. 그리고 2020년도에 가장 많이 추가가 되었다는 것도 확인할 수 있다.

### 2_2_2__한국 영화가 추가된 시기 확인

한국 영화도 빼먹을 수 없다. 한국 영화도 위와 동일한 방법으로 시각화해 본다.

In [ ]:
netflix_movies_date = netflix_rok_movies[['date_added']].dropna()
netflix_movies_date['year'] = netflix_movies_date['date_added'].apply(lambda x : x.split(', ')[-1])
netflix_movies_date['month'] = netflix_movies_date['date_added'].apply(lambda x : x.lstrip().split(' ')[0])

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'][::-1]
df = netflix_movies_date.groupby('year')['month'].value_counts().unstack().fillna(0)[month_order]

fig = px.imshow(df, labels=dict(x='', y='', color='Count'), x=df.columns, y=df.index)
fig.update_layout(title='넷플릭스 대한민국 한국 영화 업데이트 연도별 월간 추가 추세')

fig.show()

신기한 점이 있는데, 한국 영화의 경우 12월에는 추가되지 않은 듯 하다.

정말로 없는 것일까? 직접 확인해보도록 하자.

In [ ]:
netflix_movies_date['month'].unique()

정말로 없었다(<u>넷플릭스 대한민국 열일하자...</u>).

그렇다면 12월(December) 컬럼을 제외시키고 다시 돌려보도록 하자.

In [ ]:
netflix_movies_date = netflix_rok_movies[['date_added']].dropna()
netflix_movies_date['year'] = netflix_movies_date['date_added'].apply(lambda x : x.split(', ')[-1])
netflix_movies_date['month'] = netflix_movies_date['date_added'].apply(lambda x : x.lstrip().split(' ')[0])

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November'][::-1]
df = netflix_movies_date.groupby('year')['month'].value_counts().unstack().fillna(0)[month_order]

fig = px.imshow(df, labels=dict(color='Count'), x=df.columns, y=df.index)
fig.update_layout(title='넷플릭스 대한민국 한국 영화 업데이트 연도별 월간 추가 추세')

fig.show()

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
ax.set_title('넷플릭스 대한민국 한국 영화 연간 추가 추세', family='D2Coding', size=20)

plt.xkcd()    

sns.set(style='whitegrid')
sns.countplot(data=netflix_rok_movies, y='release_year',
                   palette='husl', order=netflix_rok_movies['release_year']
                   .value_counts().index[0:])

전반적으로 2018년도에 가장 많이 추가되었으며 그 중에서 2018년 2월이 한국 영화가 많이 등록된 것을 확인할 수 있다.

## 2_3_넷플릭스 대한민국 영상물(컨텐츠) 등급 확인

### 2_3_1_한국 영화 영상물 등급 확인

먼저, 모든 한국 영화의 등급을 시각화해본다.

In [ ]:
netflix_rok_movies['rating'].unique()

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
ax.set_title('넷플릭스 대한민국 한국 영화 영상물 등급', family='D2Coding', size=20)

plt.xkcd()
    
sns.set(style = "whitegrid")
sns.countplot(data=netflix_rok_movies, x='rating', palette='husl',
             order=netflix_rok_movies['rating'].value_counts().index[0:])

위의 등급은 **[영상물 등급 제도/미국](https://namu.wiki/w/%EC%98%81%EC%83%81%EB%AC%BC%20%EB%93%B1%EA%B8%89%20%EC%A0%9C%EB%8F%84/%EB%AF%B8%EA%B5%AD)**을 기준으로 구분된 것이다.

전체적으로 넷플릭스 대한민국이 수입해 오는 한국 영화들이 <u>대부분 자극적인(TV-MA: 17세 미만의 어린이 혹은 청소년한테 부적절한 프로그램)</u> 것을 확인할 수 있다.

다음으로 많은 "NR" 등급의 경우 등급 보류를 뜻하며, TV-14, TV-Y7, TV-PG의 경우 같은 결과를 보여주고 있다.

### 2_3_2_한국 TV 프로그램 영상물 등급 확인

다음으로, TV 프로그램도 위와 동일한 방법으로 시각화해 보자.

In [ ]:
netflix_rok_shows['rating'].unique()

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
ax.set_title('넷플릭스 대한민국 한국 TV 프로그램 영상물 등급', family='D2Coding', size=20)

plt.xkcd()
    
sns.set(style = "whitegrid")
sns.countplot(data=netflix_rok_shows, x='rating', palette='pastel',
             order=netflix_rok_shows['rating'].value_counts().index[0:])

<u>한국 영화와는 다르게 "TV-14(14세 미만의 어린이 혹은 청소년이 시청하려면 보호자 지도가 권장되는 프로그램)" 등급 판정을 받은 TV 프로그램이 주를 이루고 있</u>으며, 다음으로 TV-MA, TV-PG 순으로 결과를 확인할 수 있다.

## 2_4_넷플릭스 대한민국 컨텐츠 상영시간(Duration) 확인

### 2_4_1_한국 영화 상영시간 확인

In [ ]:
netflix_rok_df['duration'].unique()

In [ ]:
colors = ['lightslategray']
colors[0] = 'crimson'

movie_dur = pd.value_counts(netflix_rok_movies['duration'])
fig = go.Figure([go.Bar(x=movie_dur.index, y=movie_dur.values,
                        text=movie_dur.values, marker_color=colors)])
fig.update_traces(textposition='outside')
fig.update_layout(title='한국 영화 상영시간 분포도', xaxis={'categoryorder':'total descending'})

fig.show()

데이터가 적어서 속단하기 이르지만, 현재 넷플릭스 대한민국에 있는 한국 영화 대부분이 2시간(120분) 내외로 끝나는 것을 알 수 있었다.

### 2_4_2_한국 TV 프로그램 상영시간 확인

위와 동일한 방법으로 시각화해 보자.

In [ ]:
netflix_rok_shows['duration'].unique()

In [ ]:
colors = ['lightslategray']
colors[0] = 'crimson'

show_dur = pd.value_counts(netflix_rok_shows['duration'])
fig = go.Figure([go.Bar(x=show_dur.index, y=show_dur.values,
                        text=show_dur.values, marker_color=colors)])
fig.update_traces(textposition='outside')
fig.update_layout(title='한국 TV 프로그램 상영시간 분포도', xaxis={'categoryorder':'total descending'})

fig.show()

## 2_5_IMDb 데이터셋 추가 후 평점 비교

Kaggle에는 [IMDb 데이터셋(IMDb movies extensive dataset)](https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset)과 [TMDb 데이터셋(TMDb 5000 Movie Dataset)](https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset)가 있는데 양쪽 모두 영화 및 TV 프로그램에 대해 평점 데이터를 제공한다.

여기서 굳이 IMDb 데이터셋을 활용하는 이유는 TMDb의 경우에는 '좋은 평가'와 '좋지 않은 평가'를 투표한 사람을 퍼센트 단위로 표현하는데, IMDb는 평점을 수치화(0~10점)하기 때문에 객관적이며 보다 좋은 시각화 결과를 얻을 수 있을 것 같아서이다.

'IMDb movies extensive dataset'에는 <u>평점을 데이터로 분리한 데이터셋(IMDb ratings.csv)</u>과 <u>영화 정보 데이터를 분리한 데이터셋(IMDb movies.csv)</u>, 그리고 <u>배우 정보를 데이터로 분리한 데이터셋(IMDb names.csv)</u>으로 구분되어 있다.

우리가 해야할 것은 기존 넷플릭스 대한민국에 등록되어있는 데이터들을 기반으로 평점을 매겨야 하기 때문에 "IMDb ratings.csv" 파일과 "IMDb movies.csv" 파일을 불러온 후 시각화 작업을 하도록 한다.

In [ ]:
imdb_rate = pd.read_csv('dataset/IMDb ratings.csv',
                        usecols=['weighted_average_vote'], encoding='utf-8', engine='python')
print(imdb_rate.shape)

In [ ]:
imdb_title = pd.read_csv('dataset/IMDb movies.csv',
                         usecols=['title', 'year', 'genre'], encoding='utf-8', engine='python')
print(imdb_title.shape)

In [ ]:
# 위의 두 csv 파일을 pandas의 DataFrame 형태로 병합
rating_df = pd.DataFrame({'Title':imdb_title.title,
                       'Genre':imdb_title.genre,
                       'Release Year':imdb_title.year,
                       'Rating':imdb_rate.weighted_average_vote})
rating_df.drop_duplicates(subset=['Title', 'Rating', 'Release Year'], inplace=True)
print(rating_df.shape)

### 2_5_1_한국 영화 평점 시각화

먼저, IMDb rating 데이터셋과 넷플릭스 대한민국 데이터셋을 내부 조인(Inner join)한다.

In [ ]:
rating_df.dropna()

joint_movies_df = rating_df.merge(netflix_rok_movies, left_on='Title', right_on='title', how='inner')
joint_movies_df = joint_movies_df.sort_values(by='Rating', ascending=False)

In [ ]:
print(joint_movies_df.shape)
joint_movies_df.head()

In [ ]:
# IMDb의 평점(Rating) 상위 10개 데이터를 시각화
top_10_movies = joint_movies_df[0:20]

fig = px.sunburst(top_10_movies, path=['title', 'rating'],
                 values='Rating', color='Rating')
fig.update_layout(title='IMDb 평점 + 넷플릭스 대한민국 한국 영화')

fig.show()

In [ ]:
print(top_10_movies.shape)
top_10_movies.head(10)

### 2_5_2_TV 프로그램 평점 시각화

2_5_1과 같은 작업을 거쳐 시각화하는 과정이다.

In [ ]:
rating_df.dropna()

joint_shows_df = rating_df.merge(netflix_rok_shows, left_on='Title', right_on='title', how='inner')
joint_shows_df = joint_shows_df.sort_values(by='Rating', ascending=False)

In [ ]:
top_10_shows = joint_shows_df[0:10]

fig = px.sunburst(top_10_shows, path=['title', 'rating'],
                 values='Rating', color='Rating')
fig.update_layout(title='IMDb 평점 + 넷플릭스 대한민국 TV 프로그램')

fig.show()

In [ ]:
print(top_10_shows.shape)
top_10_shows.head(10)

## 2_6_넷플릭스 대한민국 컨텐츠 주요 장르를 WordCloud 형태로 시각화

### 2_6_1_한국 영화 장르 WordCloud

WordCloud를 사용해서 `netflix_rok_movies`의 'listed_in' 컬럼을 추출해서 이를 시각화해 보자.

In [ ]:
from collections import Counter

movies_genre = list(netflix_rok_movies['listed_in'])
mov_genre = []

for g in movies_genre:
    g = list(g.split(','))
    for i in g:
        mov_genre.append(i.replace(' ', ""))

mov_gen = Counter(mov_genre)

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

text = list(set(mov_genre))
plt.rcParams['figure.figsize'] = (11, 11)

mask = np.array(Image.open('imgs/black-star1.png'))
wc = WordCloud(max_words=1000000, background_color='black', mask=mask).generate(str(text))

plt.imshow(wc, interpolation='bilinear')
plt.axis('off')

plt.show()

### 2_6_2_한국 TV 프로그램 장르 WordCloud

위와 동일한 방법으로 추출 후 시각화해 보자.

In [ ]:
shows_genre = list(netflix_rok_shows['listed_in'])
sho_genre = []

for g in shows_genre:
    g = list(g.split(','))
    for i in g:
        sho_genre.append(i.replace(' ', ""))

sho_gen = Counter(sho_genre)

In [ ]:
text = list(set(sho_genre))
plt.rcParams['figure.figsize'] = (11, 11)

mask = np.array(Image.open('imgs/black-star1.png'))
wc = WordCloud(max_words=1000000, background_color='black', mask=mask).generate(str(text))

plt.imshow(wc, interpolation='bilinear')
plt.axis('off')

plt.show()

# 3_베이스라인(Baseline) 모델 생성 후 추천 알고리즘 구현

베이스라인 모델은 쉽게 말해 개인의 성향을 모델에 반영한 후, 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것을 일컫는다.

보통 **베이스라인 모델**은 *전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산*한다.

* **전체 평균 평점** : <u>모든 사용자의 아이템에 대한 평점을 평균한 값</u>

* **사용자 편향 점수** : <u>사용자별 아이템 평점 평균 값 - 전체 평균 평점</u>

* **아이템 편향 점수** : <u>아이템별 평점 평균 값 - 전체 평균 평점</u>

예를 들어 영화 평점을 베이스라인 모델의 평점을 고려해 이를 적용해 보자.

모든 사용자의 평균적인 영화 평점이 5점 만점 중 3.5이고(전체 평균 평점: 3.5), '나 홀로 집에 1편'을 모든 사용자가 평균적으로 평점 4.2로 평가했다면 특정 사용자인 '사용자: 민재'가 '나 홀로 집에 1편'을 어떻게 평가할 것인지를 예상해 본다면...

---

```
"모든 사용자의 평균 영화 평점: 3.5"
                +
"사용자 편향 점수: 4.0 - 3.5 = 0.5"
                +
"아이템 편향 점수: 4.2 - 3.5 = 0.7"
                
                =
                
"사용자: 민재의 '나 홀로 집에 1편' 베이스라인 모델 평점 = 4.7(3.5+0.5+0.7)"
```

---

와 같은 결과를 얻게 된다.

## 3_1_교차 검증과 하이퍼파라미터 튜닝

[Surprise Library](https://surprise.readthedocs.io/en/stable/index.html)는 교차 검증과 하이퍼파라미터 튜닝에 적합하다. Surprise는 사이킷런의 `cross_validate()` , `GridSearchCV`와 유사한 클래스를 제공하고 있다.

가장 먼저 교차 검증을 시행해 볼텐데, `cross_validate()`를 이용해 [MovieLens](https://grouplens.org/datasets/movielens/)에서 제공해주는 영화 데이터셋을 활용할 것이며, `ml_rating_df`라는 이름으로 DataFrame으로 로딩한 데이터를 5개의 학습(train) / 검증(test) 폴드 데이터셋으로 분리해 교차 검증을 수행하고, RMSE(Root Mean Square Error), MAE(Mean Absolute Error)로 성능 평가를 진행해 보겠다.

참고로 RMSE, MAE 중 어느 것을 사용해야 하는지 잘 모르겠다면, <u>[왜 직관적인 MAE 말고 RMSE를 쓰시나요 | "김문과의 데이터"](https://data101.oopy.io/mae-vs-rmse)</u> 님의 블로그를 참조하기 바란다.

In [ ]:
import pandas as pd

ml_rating = pd.read_csv('dataset/ml-latest-small/ratings.csv', usecols=['userId', 'movieId', 'rating'],
                        encoding='utf-8', engine='python')[['movieId', 'userId', 'rating']]
print(ml_rating.shape)

In [ ]:
ml_rating_df = pd.DataFrame(ml_rating, columns=ml_rating.keys())
print(ml_rating_df.shape)

In [ ]:
ml_rating_df.info()

In [ ]:
ml_rating_df.dtypes

In [ ]:
ml_rating_df.head()

In [ ]:
# 인덱스와 헤더를 모두 제거한 후 ml_rating_df_nohead.csv를 생성함.
ml_rating_df.to_csv('dataset/ml-latest-small/ml_rating_df_nohead.csv',
                    index=False, header=False)

In [ ]:
ml_rating_noh_df = pd.read_csv('dataset/ml-latest-small/ml_rating_df_nohead.csv')
print(ml_rating_noh_df.shape)
ml_rating_noh_df.head()

In [ ]:
import surprise

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise import SVD, NMF, SlopeOne, BaselineOnly

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ml_rating_df[['userId', 'movieId', 'rating']], reader)

svd = SVD(random_state=0)
nmf = NMF(random_state=0)
slope = SlopeOne()
bsl = BaselineOnly(bsl_options={'n_epochs': 20, 'reg_i': 12, 'reg_u': 13})

'''
알고리즘 객체(svd, nmf, slope, bsl), 데이터(data), 성능 평가 방법(measures),
폴드 데이터셋 개수(cv), 하나를 제외한 모든 CPU 코어 사용(n_jobs=-2)
'''
svd_surprise = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-2, verbose=False)
nmf_surprise = cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-2, verbose=False)
slope_surprise = cross_validate(slope, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-2, verbose=False)
bsl_surprise = cross_validate(bsl, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-2, verbose=False)

In [ ]:
print('# SVD Result:\n', svd_surprise)
print('\n# NMF(Non-negative Matrix Factorization) Result:\n', nmf_surprise)
print('\n# SlopeOne Result:\n', slope_surprise)
print('\n# BaselineOnly Result:\n', bsl_surprise)

In [ ]:
# import csv

# with open('result/bsl_surprise.csv', 'w') as f:
#     w = csv.writer(f)
#     w.writerow(bsl_surprise.keys())
#     w.writerow(bsl_surprise.values())

Surprise의 GridSearchCV도 사이킷런의 GridSearchCV와 유사하게 교차 검증을 통한 하이퍼파라미터 최적화를 수행한다. 알고리즘 유형에 따라 다르겠지만, SVD(Singular Value Decomposition)의 경우 주로 점진적 하강 방식(Stochastic Gradient Descent)의 반복 횟수를 지정하는 `n_epochs`와 SVD의 잠재 요인 K의 크기를 지정하는 `n_factors`를 튜닝한다.

이번 예제에서는 `'n_epochs':[20, 40, 60]` , `'n_factors':[50, 100, 200]`로 크기를 변경하면서 CV가 '3'일 때의 최적 하이퍼파라미터를 도출해 보겠다.

In [ ]:
from surprise.model_selection import GridSearchCV

# 최적화할 하이퍼파라미터를 딕셔너리 형태로 지정함
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# CV=3, 성능 평가는 RMSE, MAE로 수행함
gridSearch = GridSearchCV(NMF, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-2)
gridSearch.fit(data)

# 최고 RMSE 평가 점수와 하이퍼파라미터를 출력함
print('GridSearchCV Best Score: ', gridSearch.best_score['rmse'])
print('GridSearchCV Best Parameters: ', gridSearch.best_params['rmse'])

`'n_epochs': 20` , `'n_factors': 50`일 때 CV=3 검증 데이터셋에서 최적 RMSE가 약 0.878로 나왔다.

## 3_2_개인화 영화 추천 시스템 구축

Surprise를 이용해 잠재 요인 협업 필터링 기반의 개인화된 영화 추천을 구현해 보겠다.

지금까지는 학습 데이터로 `.fit()`을 호출해 학습을 수행하고 테스트 데이터로 `.test()`를 호출해 예측 평점을 계산한 뒤 RMSE/MSE로 성능을 평가했다.

하지만 이번에는 다른 방법을 사용할 것인데, Surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화를 추천해 보겠다.

### 3_2_1_[MovieLens](https://grouplens.org/datasets/movielens/) 데이터셋

In [ ]:
# train_test_split()로 분리되지 않는 데이터셋에 .fit()를 호출할 경우 다음과 같은 에러가 발생한다.
data = Dataset.load_from_df(ml_rating_df[['userId', 'movieId', 'rating']], reader)
algo = BaselineOnly()
algo.fit(data)

데이터셋 전체를 학습 데이터로 사용하려면 `DatasetAutoFolds` 클래스를 호출할 필요가 있다.

`DatasetAutoFolds` 객체를 생성한 후 `buiid_full_trainset()` 메서드를 호출하면 전체 데이터를 학습 데이터셋으로 만들 수 있다.

In [ ]:
from surprise.dataset import DatasetAutoFolds

# Reader 객체 생성 시 line_format 인자로 user, item, rating, timestamp의 4개의 컬럼으로 구성함
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 "rating_ml_df_nohead.csv" 파일 기반으로 생성한다.
data_folds = DatasetAutoFolds(ratings_file='dataset/ml-latest-small/ml_rating_df_nohead.csv',
                              reader=reader)

# 전체 데이터를 학습 데이터로 생성함
train_dataset = data_folds.build_full_trainset()

생성된 학습 데이터를 기반으로 학습을 수행해 보겠다. 이후에 특정 사용자가 영화를 추천하기 위해 아직 보지 않은 영화 목록을 확인해 보도록 하자.

In [ ]:
# SVD 알고리즘으로 학습을 수행함
algo = BaselineOnly()
algo.fit(train_dataset)

특정 사용자는 `userId=9`인 사용자로 저정한 후 userId 9번이 아직 평점을 매기지 않은 영화 [`movieId=1(Toy Story)`](https://movielens.org/movies/1)로 선정한 뒤 예측 평점을 계산해 본다.

In [ ]:
ml_rating_df['userId'].unique()

In [ ]:
ml_rating_df['movieId'].unique()

In [ ]:
ml_rating_df_test = ml_rating_df[ml_rating_df['userId'] == 610]
ml_rating_df_test.head()

In [ ]:
# 영화 데이터셋 DataFrame 로딩함
ml_movie = pd.read_csv('dataset/ml-latest-small/movies.csv')

# userId=610 추출 후 movieId=3 데이터가 있는지 확인함
movieIds = ml_rating_df[ml_rating_df['userId'] == 610]['movieId']

if movieIds[movieIds == 3].count() == 0:
    print('# 해당 사용자의 영화 평점 정보 없음\n')

print(ml_movie[ml_movie['movieId'] == 3])

`movieId=1`인 영화에 대해서 `userId=9` 사용자의 추천 예상 평점은 `.predict()` 메서드를 이용하면 된다.

단, 값을 입력할 때 문자열(string) 형태여야만 한다.

In [ ]:
uid = str(610)
iid = str(3)

pred = algo.predict(uid, iid, verbose=True)

추천 예측 평점 값은 "3.82"로 도출되었다.

사용자가 평점을 매기지 않은 영화의 추천 예측 평점을 구했으니 다음으로 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤 예측 평점 순으로 영화를 추천해 보겠다.

먼저, 추천 대상이 되는 영화 정보를 추출한다.

In [ ]:
def get_unseen_movie_surprise(ml_rating_df, ml_movie, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 list 형태로 생성함
    seen_movies = ml_rating_df[ml_rating_df['userId'] == userId]['movieId'].tolist()

    # 모든 영화의 movieId를 list 형태로 생성함
    total_movies = ml_movie['movieId'].tolist()

    # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 list 형태로 생성함
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('# 평점 매긴 영화의 수: ', len(seen_movies), '\n# 추천 대상 영화의 수: ', len(unseen_movies), '\n# 전체 영화의 수: ', len(total_movies))

    return unseen_movies

# 평점 DataFrame, 영화 DataFrame, userId
unseen_movies = get_unseen_movie_surprise(ml_rating_df, ml_movie, 610)

`userId=9`인 사용자의 추천 대상 영화의 수는 9696개(전체 영화의 수 - 평점 매긴 영화의 수)이며, SVD 알고리즘을 이용해 높은 예측 평점을 가진 순으로 영화를 추천해 보겠다.

보다 쉽게 추출하기 위해 새롭게 `recommend_movie_surprise()` 함수를 생성하도록 하겠다.

위의 함수는 추천 대상 영화 모두를 대상으로 추천 알고리즘 객체의 `.predict()` 메서드를 호출한 뒤 그 결과인 Prediction 객체를 list 형태로 저장하는 함수이다.

이렇게 저장된 list 내부의 Prediction 객체를 예측 평점이 높은 순으로 다시 정렬한 후 <u>TOP-N개의 Prediction 객체에서 '영화 ID', '영화 제목', '예측 평점'을 추출한 후 반환</u>한다.

In [ ]:
def recommend_movie_surprise(algo, userId, unseen_movies, top_n=10):
    '''
    알고리즘 객체의 predict() 메서드를 평점이 없는 영화에
    반복 수행한 후 결과를 list 객체로 저장함
    '''
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    '''
    predictions list 객체는 아래와 같이 surprise의 Predictions 객체를 원소로 가지고 있음.
    [Prediction(user_id='9', movie_id='1', est=3.64).....]

    이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의할 필요가 있음.
    sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행함
    '''
    def sortkey_est(pred):
        return pred.est
    
    '''
    sortkey_est( ) 반환값의 내림차순으로 정렬 수행하고
    top_n개의 최상위 값을 추출함
    '''
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출, 영화 ID, 예상 평점, 영화 제목을 추출함
    top_n_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_n_movie_rating = [pred.est for pred in top_predictions]
    top_n_movie_titles = ml_movie[ml_movie.movieId.isin(top_n_movie_ids)]['title']
    top_n_movie_preds = [(id, title, rating) for id, title, rating in zip(top_n_movie_ids, top_n_movie_titles, top_n_movie_rating)]
    
    return top_n_movie_preds

unseen_movies = get_unseen_movie_surprise(ml_rating_df, ml_movie, 610)
top_movie_preds = recommend_movie_surprise(algo, 610, unseen_movies, top_n=10)
print('\n### 해당 사용자를 위한 Top10 추천 영화 리스트 ###')

for top_movie in top_movie_preds:
    print("'title':", top_movie[1], "'estimate_score':", top_movie[2])

### 3_2_2_[넷플릭스 대한민국](https://www.netflix.com/kr/) 개인화 영화 추천 시스템 구축

3_2_1에서 사용했던 'MovieLens' 데이터셋을 '넷플릭스 대한민국' 데이터셋에 조인(Join)을 해야 하는데, 넷플릭스 데이터셋과 다르며(한국 영화 외에도 다양한 국가의 영화 정보가 담겨있음), 특정 'movieId'의 국가 정보를 확인할 수 있는 컬럼이 존재하지 않으므로 이를 해결할 방안을 고민 중이다.

---

<u>**! 방법이 있긴 하다.**</u>

1) <u>MovieLens 데이터셋</u>의 "links.csv"에는 'imdbId' 컬럼이 있으며, <u>IMDb 데이터셋</u>의 "IMDb movies.csv"에는 'imdb_title_id' 컬럼이 있다.

2) 2_5_부분에서 <u>IMDb 데이터셋</u>과 <u>넷플릭스 데이터셋</u>을 내부 조인(Inner join)하기 전에 `pd.read_csv()` 함수 중 `usecols=['imdb_title_id']`만 사용을 한다.

3) "IMDb movies.csv"의 'imdb_title_id' 컬럼명을 "links.csv"의 'imdbId'로 `pd.DataFrame()` 형태로 변경한다.

4) 새로운 이름의 DataFrame이 생성되었으면 넷플릭스 데이터셋과 내부 조인을 한다.

5) 이후 3_2_1_의 방법대로 넷플릭스 대한민국 데이터셋을 기반으로 개인화 영화 추천 시스템을 구축한다.

In [ ]:
imdb_movies_movieId = pd.read_csv('dataset/IMDb movies.csv', usecols=['imdb_title_id'],
                                  encoding='utf-8', engine='python')
print(imdb_movies_movieId.shape)
imdb_movies_movieId.head()

In [ ]:
# 'imdb_title_id' 컬럼에 있는 문자열 "tt" 제거를 위한 함수 정의
def imdb_title_id_cleaning(x):
    if str(x).find('tt') != -1:
        return str(x).split('tt')[1].replace('', '')
    return str(x)

In [ ]:
imdb_movies_movieId_df = pd.DataFrame({'imdbId':imdb_movies_movieId.imdb_title_id})
imdb_movies_movieId_df['imdbId'] = imdb_movies_movieId_df['imdbId'].apply(imdb_title_id_cleaning).apply(lambda x:x.replace('', ''))
print(imdb_movies_movieId_df.shape)
imdb_movies_movieId_df.head()

In [ ]:
imdb_movies_movieId_df.dtypes

In [ ]:
ml_links_movieId = pd.read_csv('dataset/ml-latest/links.csv', usecols=['movieId', 'imdbId'],
                               dtype={'movieId':'object', 'imdbId':'object'}, encoding='utf-8', engine='python')
print(ml_links_movieId.shape)
ml_links_movieId.head()

In [ ]:
ml_links_movieId_df = pd.DataFrame(ml_links_movieId, columns=ml_links_movieId.keys())
print(ml_links_movieId_df.shape)
ml_links_movieId_df.head()

In [ ]:
ml_links_movieId_df.dtypes

In [ ]:
joint_imdb_ml_links_df = pd.merge(ml_links_movieId_df, imdb_movies_movieId_df, on='imdbId', how='left')
joint_imdb_ml_links_df

---

In [ ]:
ml_ratings_large = pd.read_csv('dataset/ml-latest/ratings.csv', usecols=['userId', 'movieId', 'rating'],
                               dtype={'userId':'object', 'movieId':'object', 'rating':'object'},
                               encoding='utf-8', engine='python')
print(ml_ratings_large.shape)
ml_ratings_large.head()

In [ ]:
ml_ratings_large_df = pd.DataFrame(ml_ratings_large, columns=ml_ratings_large.keys())
print(ml_ratings_large_df.shape)
ml_ratings_large_df.head()

In [ ]:
ml_ratings_large_df.dtypes

In [ ]:
joint_ml_ratings_links_df = pd.merge(joint_imdb_ml_links_df, ml_ratings_large_df, on='movieId', how='right')
print(joint_ml_ratings_links_df.shape)
joint_ml_ratings_links_df

In [ ]:
joint_ml_ratings_links_df.isnull().sum()

In [ ]:
joint_ml_ratings_links_df.dtypes

---

In [ ]:
# ml-latest 'movies.csv' 데이터셋의 'title' 컬럼에 있는 문자열 "(????)" 제거를 위한 함수 정의
def title_cleaning(x):
    if str(x).find('(') != -1:
        return str(x).split('(')[0].replace(')', '')
    return str(x)

In [ ]:
ml_movies_large = pd.read_csv('dataset/ml-latest/movies.csv',
                              usecols=['movieId', 'title'],
                              dtype={'movieId':'object',
                                     'title':'object'},
                              encoding='utf-8', engine='python')
ml_movies_large['title'] = ml_movies_large['title'].apply(title_cleaning).apply(lambda x:x.replace('', ''))
ml_movies_large

In [ ]:
ml_movies_large.dtypes

In [ ]:
ml_movies_large_df = pd.DataFrame(ml_movies_large, columns=ml_movies_large.keys())
print(ml_movies_large_df.shape)

In [ ]:
joint_ml_all_df = pd.merge(ml_movies_large_df, joint_ml_ratings_links_df, on='movieId', how='right')
joint_ml_all_df

In [ ]:
joint_ml_all_df.isnull().sum()

In [ ]:
joint_ml_all_df.dtypes

In [ ]:
# joint_ml_all_df.to_csv('dataset/ml-latest/all.csv')

In [ ]:
ml_all = pd.read_csv('dataset/ml-latest/all.csv',
                     usecols=['title', 'userId', 'rating'],
                     dtype={'title':'object', 'userId':'object', 'rating':'object'},
                     encoding='utf-8', engine='python')
print(ml_all.shape)
ml_all.head()

In [ ]:
ml_all.dtypes

In [ ]:
ml_all_df = pd.DataFrame(ml_all, columns=ml_all.keys())
print(ml_all_df.shape)

---

In [ ]:
joint_netflix_movie_df = pd.merge(netflix_rok_movies, ml_all_df, on='title', how='left')
joint_netflix_movie_df.head()

In [ ]:
joint_netflix_movie_df.isnull().sum()

---

In [ ]:
netflix_shows_df = netflix_df[netflix_df['type'] == 'TV Show']
print(netflix_shows_df.shape)

In [ ]:
netflix_shows_df = pd.DataFrame(netflix_shows_df, columns=netflix_shows_df.keys())

In [ ]:
netflix_movies_df = netflix_df[netflix_df['type'] == 'Movie']
print(netflix_movies_df.shape)

In [ ]:
netflix_movies_df = pd.DataFrame(netflix_movies_df, columns=netflix_movies_df.keys())

In [ ]:
netflix_movies_df.isnull().sum()

In [ ]:
netflix_movies_df.count()

In [ ]:
netflix_shows_df.isnull().sum()

In [ ]:
netflix_shows_df.count()

---

In [ ]:
joint_netflix_movie_df = pd.merge(netflix_movies_df, ml_all_df, on='title', how='inner')
joint_netflix_movie_df.head()

In [ ]:
joint_netflix_movie_df.isnull().sum()

In [ ]:
joint_netflix_movie_df.count()

In [ ]:
joint_netflix_movie_df.to_csv('dataset/netflix_movies.csv')

---

In [ ]:
joint_netflix_show_df = pd.merge(netflix_shows_df, ml_all_df, on='title', how='inner')
joint_netflix_show_df.head()

In [ ]:
joint_netflix_show_df.isnull().sum()

In [ ]:
joint_netflix_show_df.count()

In [ ]:
joint_netflix_show_df.to_csv('dataset/netflix_shows.csv')

---

## ?_넷플릭스 영화 데이터 클렌징

In [ ]:
# 'netflix_movies.csv' 데이터셋의 'show_id' 컬럼에 있는 문자열 "s" 제거를 위한 함수 정의
def showId_cleaning(x):
    if str(x).find('s') != -1:
        return str(x).split('s')[1].replace('', '')
    return str(x)

In [ ]:
netflix_movies_recomm = pd.read_csv('dataset/netflix_movies.csv',
                                   usecols=['show_id', 'userId', 'rating_y'],
                                  encoding='utf-8', engine='python')
netflix_movies_recomm['show_id'] = netflix_movies_recomm['show_id'].apply(showId_cleaning).apply(lambda x:x.replace('', ''))
print(netflix_movies_recomm.shape)
netflix_movies_recomm.head()

In [ ]:
netflix_movies_recomm.dtypes

In [ ]:
netflix_movies_recomm_df = pd.DataFrame({'itemId':netflix_movies_recomm.show_id,
                                         'userId':netflix_movies_recomm.userId,
                                         'rating':netflix_movies_recomm.rating_y})

In [ ]:
netflix_movies_recomm_df.info()

In [ ]:
netflix_movies_recomm_df[['itemId']] = netflix_movies_recomm_df[['itemId']].apply(pd.to_numeric)

In [ ]:
netflix_movies_recomm_df.info()

In [ ]:
netflix_movies_recomm_df.to_csv('dataset/netflix_movies_recomm.csv')

In [ ]:
netflix_movies_recomm_df.to_csv('dataset/netflix_movies_recomm_nohead.csv',
                               index=False, header=False)

In [ ]:
netflix_movies_info = pd.read_csv('dataset/netflix_movies.csv',
                                   usecols=['show_id', 'title', 'listed_in'],
                                  encoding='utf-8', engine='python')
netflix_movies_info['show_id'] = netflix_movies_info['show_id'].apply(showId_cleaning).apply(lambda x:x.replace('', ''))
print(netflix_movies_info.shape)
netflix_movies_info

In [ ]:
netflix_movies_info.info()

In [ ]:
netflix_movies_info_df = pd.DataFrame({'itemId':netflix_movies_info.show_id,
                                       'title':netflix_movies_info.title,
                                       'genres':netflix_movies_info.listed_in})
netflix_movies_info_df.drop_duplicates(subset=['title'], inplace=True)
print(netflix_movies_info_df.shape)
netflix_movies_info_df

In [ ]:
netflix_movies_info_df[['itemId']] = netflix_movies_info_df[['itemId']].apply(pd.to_numeric)

In [ ]:
netflix_movies_info_df.info()

In [ ]:
netflix_movies_info_df.to_csv('dataset/netflix_movies_info.csv',
                             index=False)

In [ ]:
netflix_movies_info_df.to_csv('dataset/netflix_movies_info_nohead.csv',
                             index=False, header=False)

## ?_넷플릭스 TV 프로그램 데이터 클렌징

In [ ]:
# 'netflix_shows.csv' 데이터셋의 'show_id' 컬럼에 있는 문자열 "s" 제거를 위한 함수 정의
def showId_cleaning(x):
    if str(x).find('s') != -1:
        return str(x).split('s')[1].replace('', '')
    return str(x)

In [ ]:
netflix_shows_recomm = pd.read_csv('dataset/netflix_shows.csv',
                                   usecols=['show_id', 'userId', 'rating_y'],
                                   encoding='utf-8', engine='python')
netflix_shows_recomm['show_id'] = netflix_shows_recomm['show_id'].apply(showId_cleaning).apply(lambda x:x.replace('', ''))
print(netflix_shows_recomm.shape)
netflix_shows_recomm.head()

In [ ]:
netflix_shows_recomm.dtypes

In [ ]:
netflix_shows_recomm_df = pd.DataFrame({'itemId':netflix_shows_recomm.show_id,
                                        'userId':netflix_shows_recomm.userId,
                                        'rating':netflix_shows_recomm.rating_y})

In [ ]:
netflix_shows_recomm_df.info()

In [ ]:
netflix_shows_recomm_df[['itemId']] = netflix_shows_recomm_df[['itemId']].apply(pd.to_numeric)

In [ ]:
netflix_shows_recomm_df.info()

In [ ]:
netflix_shows_recomm_df.to_csv('dataset/netflix_shows_recomm.csv',
                              index=False)

In [ ]:
netflix_shows_recomm_df.to_csv('dataset/netflix_shows_recomm_nohead.csv',
                              index=False, header=False)

In [ ]:
netflix_shows_info = pd.read_csv('dataset/netflix_shows.csv',
                                   usecols=['show_id', 'title', 'listed_in'],
                                  encoding='utf-8', engine='python')
netflix_shows_info['show_id'] = netflix_shows_info['show_id'].apply(showId_cleaning).apply(lambda x:x.replace('', ''))
print(netflix_shows_info.shape)
netflix_shows_info

In [ ]:
netflix_shows_info.info()

In [ ]:
netflix_shows_info_df = pd.DataFrame({'itemId':netflix_shows_info.show_id,
                                       'title':netflix_shows_info.title,
                                       'genres':netflix_shows_info.listed_in})
netflix_shows_info_df.drop_duplicates(subset=['title'], inplace=True)
print(netflix_shows_info_df.shape)
netflix_shows_info_df

In [ ]:
netflix_shows_info_df[['itemId']] = netflix_shows_info_df[['itemId']].apply(pd.to_numeric)

In [ ]:
netflix_shows_info_df.info()

In [ ]:
netflix_shows_info_df.to_csv('dataset/netflix_shows_info.csv',
                            index=False)

In [ ]:
netflix_shows_info_df.to_csv('dataset/netflix_shows_info_nohead.csv',
                            index=False, header=False)

---

## ?_넷플릭스 영화 개인화 추천 시스템 구축

In [ ]:
import surprise

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise import SVD

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(netflix_movies_recomm_df[['itemId', 'userId', 'rating']], reader)

algo = SVD(random_state=0)

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-2, verbose=True)

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

gridSearch = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-2)
gridSearch.fit(data)

print('GridSearchCV Best Score: ', gridSearch.best_score['rmse'])
print('GridSearchCV Best Parameters: ', gridSearch.best_params['rmse'])

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='item user rating', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='dataset/netflix_movies_recomm_nohead.csv',
                              reader=reader)

train_dataset = data_folds.build_full_trainset()

In [ ]:
algo = SVD(n_epochs=40, n_factors=50, random_state=0)
algo.fit(train_dataset)

In [ ]:
# 해당 데이터셋에서 가장 많이 투표한 '123100'번 사용자를 적용함
netflix_movies_recomm_df['userId'].value_counts(sort=True)

In [ ]:
netflix_movies_recomm_df['itemId'].value_counts(sort=True)

In [ ]:
netflix_movies_recomm_test = netflix_movies_recomm_df[netflix_movies_recomm_df['userId'] == 123100]
netflix_movies_recomm_test.head()

In [ ]:
netflix_movies = pd.read_csv('dataset/netflix_movies_info.csv')

itemIds = netflix_movies_recomm_df[netflix_movies_recomm_df['userId'] == 123100]['itemId']

if itemIds[itemIds == 3939].count() == 0:
    print('# 해당 사용자의 영화 평점 정보 없음\n')

print(netflix_movies[netflix_movies['itemId'] == 3939][:1])

In [ ]:
uid = str(123100)  # userId
iid = str(3939)  # itemId

pred = algo.predict(uid, iid, verbose=True)

In [ ]:
def get_unseen_movie_surprise(netflix_movies_recomm_df, netflix_movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 list 형태로 생성함
    seen_movies = netflix_movies_recomm_df[netflix_movies_recomm_df['userId'] == userId]['itemId'].tolist()

    # 모든 영화의 movieId를 list 형태로 생성함
    total_movies = netflix_movies['itemId'].tolist()

    # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 list 형태로 생성함
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('# 평점 매긴 영화의 수: ', len(seen_movies), '\n# 추천 대상 영화의 수: ', len(unseen_movies), '\n# 전체 영화의 수: ', len(total_movies))

    return unseen_movies

# 평점 DataFrame, 영화 DataFrame, userId(object)
unseen_movies = get_unseen_movie_surprise(netflix_movies_recomm_df, netflix_movies, 123100)

In [ ]:
def recommend_movie_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(itemId)) for itemId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    top_n_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_n_movie_rating = [pred.est for pred in top_predictions]
    top_n_movie_titles = netflix_movies[netflix_movies.itemId.isin(top_n_movie_ids)]['title']
    top_n_movie_preds = [(id, title, rating) for id, title, rating in zip(top_n_movie_ids, top_n_movie_titles, top_n_movie_rating)]
    
    return top_n_movie_preds

unseen_movies = get_unseen_movie_surprise(netflix_movies_recomm_df, netflix_movies, 123100)
top_movie_preds = recommend_movie_surprise(algo, 123100, unseen_movies, top_n=10)
print('\n### 해당 사용자를 위한 Top10 추천 영화 리스트 ###')

for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])

가장 이상한 점은 <u>전체 영화의 수가 "19개" 밖에 없다는 점</u>이다. 이는 예상컨데 'MovieLens' 데이터셋에 있는 영화 제목과 'IMDb' 데이터셋에 있는 영화 제목, 마지막으로 '넷플릭스' 데이터셋에 있는 영화 제목이 미세한 차이가 있어서 발생한 문제로 보인다.

데이터를 클렌징하는 과정에서 초보적인 실수를 범한 셈이다.

## ?_넷플릭스 TV 프로그램 개인화 추천 시스템 구축

In [ ]:
import surprise

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise import SVD

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(netflix_shows_recomm_df[['itemId', 'userId', 'rating']], reader)

algo = SVD(random_state=0)

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-2, verbose=True)

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

gridSearch = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-2)
gridSearch.fit(data)

print('GridSearchCV Best Score: ', gridSearch.best_score['rmse'])
print('GridSearchCV Best Parameters: ', gridSearch.best_params['rmse'])

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='item user rating', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='dataset/netflix_shows_recomm_nohead.csv',
                              reader=reader)

train_dataset = data_folds.build_full_trainset()

In [ ]:
algo = SVD(n_epochs=40, n_factors=200, random_state=0)
algo.fit(train_dataset)

In [ ]:
# 해당 데이터셋에서 가장 많이 투표한 '123100'번 사용자를 적용함
netflix_shows_recomm_df['userId'].value_counts(sort=True)

In [ ]:
netflix_shows_recomm_df['itemId'].value_counts(sort=True)

In [ ]:
netflix_shows_recomm_test = netflix_shows_recomm_df[netflix_shows_recomm_df['userId'] == 204546]
netflix_shows_recomm_test.head()

In [ ]:
netflix_shows = pd.read_csv('dataset/netflix_shows_info.csv')

itemIds = netflix_shows_recomm_df[netflix_shows_recomm_df['userId'] == 204546]['itemId']

if itemIds[itemIds == 6231].count() == 0:
    print('# 해당 사용자의 영화 평점 정보 없음\n')

print(netflix_shows[netflix_shows['itemId'] == 6231])

In [ ]:
uid = str(204546)  # userId
iid = str(6231)  # itemId

pred = algo.predict(uid, iid, verbose=True)

In [ ]:
def get_unseen_show_surprise(netflix_shows_recomm_df, netflix_shows, userId):
    seen_shows = netflix_shows_recomm_df[netflix_shows_recomm_df['userId'] == userId]['itemId'].tolist()

    total_shows = netflix_shows['itemId'].tolist()

    unseen_shows = [show for show in total_shows if show not in seen_shows]
    print('# 평점 매긴 영화의 수: ', len(seen_shows), '\n# 추천 대상 영화의 수: ', len(unseen_shows), '\n# 전체 영화의 수: ', len(total_shows))

    return unseen_shows

unseen_shows = get_unseen_show_surprise(netflix_shows_recomm_df, netflix_shows, 204546)

In [ ]:
def recommend_show_surprise(algo, userId, unseen_shows, top_n=10):
    predictions = [algo.predict(str(userId), str(itemId)) for itemId in unseen_shows]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    top_n_show_ids = [int(pred.iid) for pred in top_predictions]
    top_n_show_rating = [pred.est for pred in top_predictions]
    top_n_show_titles = netflix_shows[netflix_shows.itemId.isin(top_n_show_ids)]['title']
    top_n_show_preds = [(id, title, rating) for id, title, rating in zip(top_n_show_ids, top_n_show_titles, top_n_show_rating)]
    
    return top_n_show_preds

unseen_shows = get_unseen_show_surprise(netflix_shows_recomm_df, netflix_shows, 204546)
top_show_preds = recommend_show_surprise(algo, 204546, unseen_shows, top_n=10)
print('\n### 해당 사용자를 위한 Top10 추천 영화 리스트 ###')

for top_show in top_show_preds:
    print(top_show[1], ':', top_show[2])

---

## 3_3_넷플릭스 장르 속성을 활용한 사용자 컨텐츠 추천 시스템 구축

컨텐츠 기반의 필터링은 <u>사용자가 특정 영화를 감상하고 해당 컨텐츠가 마음에 들었으면 해당 컨텐츠와 유사한 속성/특성, 구성 요소 등을 가진 다른 컨텐츠를 추천하는 것이 목적</u>이다.

예시를 들자면, 사용자(고객)가 영화 '인셉션(Inception)'를 재밌게 봤다면 영화 '인셉션'의 장르인 "액션", "공상 과학(sci-fi)", "판타지", "스릴러"로 높은 평점을 받은 다른 영화를 추천하거나 감독인 '크리스토퍼 놀란'의 다른 영화를 추천하는 방식이다.

이처럼 *컨텐츠(또는 서비스/상품 등) 간의 유사성을 판단하는 기준이 컨텐츠 기반 필터링*이다.

In [1]:
import pandas as pd

netflix = pd.read_csv('dataset/netflix_titles.csv')
print(netflix.shape)
netflix.head(1)

(7787, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...


In [2]:
tmdb = pd.read_csv('dataset/tmdb_5000_movies.csv')
print(tmdb.shape)
tmdb.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
# movie_poster = pd.read_csv('dataset/movies_metadata.csv')
# print(movie_poster.shape)
# movie_poster.head(1)

In [ ]:
# movie_poster.info()

In [3]:
tmdb_sel_df = tmdb[['title', 'vote_average', 'vote_count']]

In [4]:
netflix_sel_df = netflix[['show_id', 'type', 'title', 'director', 'cast', 'listed_in', 'description']]

In [5]:
netflix_sel_df = pd.DataFrame({'id':netflix.show_id,
                               'type':netflix.type,
                               'title':netflix.title,
                               'director':netflix.director,
                               'cast':netflix.cast,
                               'genres':netflix.listed_in,
                               'overview':netflix.description})
netflix_sel_df.head(3)

,id,type,title,director,cast,genres,overview
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...","International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...","Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...","Horror Movies, International Movies","When an army recruit is found dead, his fellow..."


In [6]:
netflix_sel_df['cast'] = netflix_sel_df['cast'].apply(lambda x: [x])
netflix_sel_df['genres'] = netflix_sel_df['genres'].apply(lambda x: [x])
netflix_sel_df.head(3)

,id,type,title,director,cast,genres,overview
0,s1,TV Show,3%,NaN,"[João Miguel, Bianca Comparato, Michel Gomes, ...","[International TV Shows, TV Dramas, TV Sci-Fi ...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"[Demián Bichir, Héctor Bonilla, Oscar Serrano,...","[Dramas, International Movies]",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"[Tedd Chan, Stella Chung, Henley Hii, Lawrence...","[Horror Movies, International Movies]","When an army recruit is found dead, his fellow..."


In [ ]:
# movie_poster_df = movie_poster[['title', 'poster_path']]

In [7]:
netflix_tmdb_df = pd.merge(netflix_sel_df, tmdb_sel_df, on='title', how='inner')
netflix_tmdb_df.head(3)

,id,type,title,director,cast,genres,overview,vote_average,vote_count
0,s4,Movie,9,Shane Acker,"[Elijah Wood, John C. Reilly, Jennifer Connell...","[Action & Adventure, Independent Movies, Sci-F...","In a postapocalyptic world, rag-doll robots hi...",6.6,1262
1,s5,Movie,21,Robert Luketic,"[Jim Sturgess, Kevin Spacey, Kate Bosworth, Aa...",[Dramas],A brilliant group of students become card-coun...,6.5,1375
2,s45,Movie,Æon Flux,Karyn Kusama,"[Charlize Theron, Marton Csokas, Jonny Lee Mil...","[Action & Adventure, Sci-Fi & Fantasy]","Aiming to hasten an uprising, the leader of an...",5.4,703


In [9]:
# netflix_tmdb_df.to_csv('dataset/netflix_tmdb.csv', index=False)

In [ ]:
# netflix_tmdb_poster_df = pd.merge(netflix_tmdb_df, movie_poster_df, on='title', how='inner')
# netflix_tmdb_poster_df.head(3)

In [ ]:
# netflix_tmdb_poster_df.drop_duplicates(subset=['id'], inplace=True)

In [ ]:
# netflix_tmdb_poster_df.to_csv('dataset/netflix_tmdb_poster.csv')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백 문자로 단어가 구분되는 문자열로 변환한다.
netflix_tmdb_df['genres_literal'] = netflix_tmdb_df['genres'].apply(lambda x:(' ').join(x))
cvt = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_matrix = cvt.fit_transform(netflix_tmdb_df['genres_literal'])

print(genre_matrix.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

genre_similar = cosine_similarity(genre_matrix, genre_matrix)

print(genre_similar.shape)
print(genre_similar[:2])

In [ ]:
genre_similar_sorted_idx = genre_similar.argsort()[:, ::-1]

print(genre_similar_sorted_idx[:1])

In [ ]:
def find_similar_movie(find_movie_df, sorted_idx, title_name, top_n=10):
    # 인자로 입력된 netflix_tmdb_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame을 추출함
    title_movie = find_movie_df[find_movie_df['title'] == title_name]
    
    '''
    title_name을 가진 DataFrame의 인덱스 객체를 ndarray 형태로 반환하고 sorted_idx 인자로 입력된
    genre_similar_sorted_idx 객체에서 유사도 순으로 top_n개의 인덱스를 추출함
    '''
    title_idx = title_movie.index.values
    similar_idxes = sorted_idx[title_idx, :(top_n)]
    
    '''
    추출된 top_n 인덱스를 출력한다. 2차원 데이터이며,
    DataFrame에서 인덱스로 사용하기 위해서 1차원 array로 변경한다.
    '''
    print(similar_idxes)
    similar_idxes = similar_idxes.reshape(-1)
    
    return find_movie_df.iloc[similar_idxes]

In [ ]:
pd.set_option('max_colwidth', 150)  # DataFrame의 컬럼 너비를 조절함

similar_movies = find_similar_movie(netflix_tmdb_df, genre_similar_sorted_idx, 'Inception', 10)
similar_movies[['title', 'genres', 'vote_average']].sort_values('vote_average', ascending=False)

In [ ]:
netflix_tmdb_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

In [ ]:
# Pandas의 Series 객체의 quantile() 함수로 상위 70% 값을 추출함
C = netflix_tmdb_df['vote_average'].mean()
m = netflix_tmdb_df['vote_count'].quantile(0.7)

print('# 전체 영화에 대한 평균 평점(C):', round(C, 3), '\n# 평점을 부여하기 위한 최소 투표 횟수(m):', round(m, 3))

In [ ]:
'''
레코드의 인자로 받아 레코드의 'vote_count'와 'vote_average' 컬럼,
그리고 미리 추출된 C와 m 값을 적용해 레코드별 가중 평점을 반환한다.
'''
m = netflix_tmdb_df['vote_count'].quantile(0.7)
C = netflix_tmdb_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )

netflix_tmdb_df['weighted_vote_rating'] = netflix_tmdb_df.apply(weighted_vote_average, axis=1)

In [ ]:
# 상위 10개 영화를 추출함
netflix_tmdb_df[['title', 'genres', 'vote_average', 'weighted_vote_rating', 'vote_count']].sort_values(
    'weighted_vote_rating', ascending=False)

In [ ]:
# netflix_tmdb_poster_df.to_csv('result/test.csv')

1위를 차지한 영화 '펄프 픽션 (1994)'은 같은해 프랑스 칸 국제 영화제에서 황금 종려상을 받은 유명한 영화라고 한다. 2위 영화 '쉰들러 리스트 (1993)'와 3위인 영화 '인셉션 (2010)'은 매우 뛰어난 영화임에 틀림없을 것으로 예상된다.

위에서 새롭게 정의된 평점 기준에 따라서 영화를 추천해 보겠다. 장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군을 선정한 후 'weighted_vote_rating' 컬럼값이 높은 순으로 top_n만큼 추출하는 방식이다.

In [ ]:
netflix_tmdb_df['title'].unique()

In [ ]:
test_df = netflix_tmdb_df[netflix_tmdb_df['title'] == 'Indiana Jones and the Temple of Doom']
test_df

In [ ]:
def find_similar_movie(find_movie_df, sorted_idx, title_name, top_n=10):
    title_movie = find_movie_df[find_movie_df['title'] == title_name]
    title_idx = title_movie.index.values
    
    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스를 추출함
    similar_idxes = sorted_idx[title_idx, :(top_n*2)]
    similar_idxes = similar_idxes.reshape(-1)
    
    # 기준 영화의 인덱스는 제외시킴
    similar_idxes = similar_idxes[similar_idxes != title_idx]
    print(similar_idxes)
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote_rating가 높은 순으로 top_n만큼 추출함
    return find_movie_df.iloc[similar_idxes].sort_values('weighted_vote_rating', ascending=False)[:top_n]

similar_movies = find_similar_movie(netflix_tmdb_df, genre_similar_sorted_idx, 'Indiana Jones and the Temple of Doom', 10)
similar_movies[['title', 'genres', 'vote_average', 'weighted_vote_rating']]

In [ ]:
similar_movies.to_csv('result/code_similar_movies.csv')

In [ ]:
movie_poster = pd.read_csv('dataset/movies_metadata.csv',
                          usecols=['title', 'poster_path'])
movie_poster.head(3)

In [ ]:
movie_poster[['title', 'poster_path']]

In [ ]:
similar_movies_poster = pd.merge(similar_movies, movie_poster, on='title')
similar_movies_poster

In [ ]:
similar_movies_poster.drop_duplicates(subset=['id'], inplace=True)
similar_movies_poster

In [ ]:
similar_movies_poster.to_csv('result/code_similar_movies_poster.csv')

앞서 추천되었던 영화하고 상당 부분 유사한 영화들이 추천된 것을 확인할 수 있다. 하지만 장르만으로 영화를 추천하는 것에는 한계가 있다고 본다. 보통 좋아하는 영화배우나 감독을 보고 영화를 보는 경우가 더 많기 때문이다. 